In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
from sklearn.decomposition import TruncatedSVD
import numpy as np

In [3]:
import pandas as pd
data = []
for i in range(1, 4):
    data.append(pd.read_csv(f"articles{i}.csv"))
data = pd.concat(data, ignore_index=True)
data

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...,...,...,...,...,...,...,...,...
142565,146028,218078,An eavesdropping Uber driver saved his 16-year...,Washington Post,Avi Selk,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Uber driver Keith Avila picked up a p...
142566,146029,218079,Plane carrying six people returning from a Cav...,Washington Post,Sarah Larimer,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Crews on Friday continued to search L...
142567,146030,218080,After helping a fraction of homeowners expecte...,Washington Post,Renae Merle,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,When the Obama administration announced a...
142568,146031,218081,"Yes, this is real: Michigan just banned bannin...",Washington Post,Chelsea Harvey,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,This story has been updated. A new law in...


In [4]:
df = pd.read_csv('data.csv', header=None, names=['title', 'article'])
df

,title,article
0,House Republicans Fret About Winning Their Hea...,washington congression republican new fear com...
1,Rift Between Officers and Residents as Killing...,bullet shell get counted blood dri votiv candl...
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",walt disney bambi open one thousand nine hundr...
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",death may great equalizer isnt necessarili eve...
4,Kim Jong-un Says North Korea Is Preparing to T...,seoul south korea north korea leader kim said ...
...,...,...
142565,An eavesdropping Uber driver saved his 16-year...,uber driver keith avila pick passeng look like...
142566,Plane carrying six people returning from a Cav...,crew friday continu search lake eri plane carr...
142567,After helping a fraction of homeowners expecte...,obama administr announc massiv effort help dis...
142568,"Yes, this is real: Michigan just banned bannin...",stori updated new law michigan prohibit local ...


In [5]:
with open('ids.txt', 'r') as reader:
    ids = reader.readlines()
ids = [int(x) for x in ids]
ids

[17283,
 17284,
 17285,
 17286,
 17287,
 17288,
 17289,
 17290,
 17291,
 17292,
 17293,
 17294,
 17295,
 17296,
 17297,
 17298,
 17300,
 17301,
 17302,
 17303,
 17305,
 17306,
 17307,
 17308,
 17309,
 17311,
 17312,
 17313,
 17314,
 17317,
 17318,
 17319,
 17321,
 17323,
 17324,
 17325,
 17326,
 17327,
 17328,
 17330,
 17331,
 17332,
 17333,
 17334,
 17335,
 17336,
 17337,
 17338,
 17339,
 17340,
 17341,
 17342,
 17344,
 17345,
 17346,
 17347,
 17348,
 17349,
 17350,
 17352,
 17353,
 17354,
 17355,
 17356,
 17358,
 17360,
 17361,
 17362,
 17363,
 17364,
 17365,
 17366,
 17367,
 17368,
 17369,
 17370,
 17371,
 17373,
 17374,
 17376,
 17378,
 17379,
 17381,
 17382,
 17383,
 17384,
 17385,
 17386,
 17387,
 17388,
 17389,
 17390,
 17391,
 17392,
 17393,
 17394,
 17395,
 17396,
 17397,
 17398,
 17399,
 17400,
 17401,
 17402,
 17403,
 17404,
 17405,
 17406,
 17409,
 17410,
 17411,
 17412,
 17413,
 17414,
 17415,
 17416,
 17417,
 17418,
 17419,
 17420,
 17421,
 17422,
 17423,
 17425,
 17426,


In [6]:
df['id'] = ids
df

,title,article,id
0,House Republicans Fret About Winning Their Hea...,washington congression republican new fear com...,17283
1,Rift Between Officers and Residents as Killing...,bullet shell get counted blood dri votiv candl...,17284
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",walt disney bambi open one thousand nine hundr...,17285
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",death may great equalizer isnt necessarili eve...,17286
4,Kim Jong-un Says North Korea Is Preparing to T...,seoul south korea north korea leader kim said ...,17287
...,...,...,...
142565,An eavesdropping Uber driver saved his 16-year...,uber driver keith avila pick passeng look like...,218078
142566,Plane carrying six people returning from a Cav...,crew friday continu search lake eri plane carr...,218079
142567,After helping a fraction of homeowners expecte...,obama administr announc massiv effort help dis...,218080
142568,"Yes, this is real: Michigan just banned bannin...",stori updated new law michigan prohibit local ...,218081


In [9]:
import numpy as np
df = df.replace(np.nan, '', regex=True)
df

,title,article,id,content
0,House Republicans Fret About Winning Their Hea...,washington congression republican new fear com...,17283,WASHINGTON — Congressional Republicans have...
1,Rift Between Officers and Residents as Killing...,bullet shell get counted blood dri votiv candl...,17284,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",walt disney bambi open one thousand nine hundr...,17285,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",death may great equalizer isnt necessarili eve...,17286,"Death may be the great equalizer, but it isn’t..."
4,Kim Jong-un Says North Korea Is Preparing to T...,seoul south korea north korea leader kim said ...,17287,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...,...,...
142565,An eavesdropping Uber driver saved his 16-year...,uber driver keith avila pick passeng look like...,218078,Uber driver Keith Avila picked up a p...
142566,Plane carrying six people returning from a Cav...,crew friday continu search lake eri plane carr...,218079,Crews on Friday continued to search L...
142567,After helping a fraction of homeowners expecte...,obama administr announc massiv effort help dis...,218080,When the Obama administration announced a...
142568,"Yes, this is real: Michigan just banned bannin...",stori updated new law michigan prohibit local ...,218081,This story has been updated. A new law in...


In [7]:
df['content'] = data['content']
df

,title,article,id,content
0,House Republicans Fret About Winning Their Hea...,washington congression republican new fear com...,17283,WASHINGTON — Congressional Republicans have...
1,Rift Between Officers and Residents as Killing...,bullet shell get counted blood dri votiv candl...,17284,"After the bullet shells get counted, the blood..."
2,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",walt disney bambi open one thousand nine hundr...,17285,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",death may great equalizer isnt necessarili eve...,17286,"Death may be the great equalizer, but it isn’t..."
4,Kim Jong-un Says North Korea Is Preparing to T...,seoul south korea north korea leader kim said ...,17287,"SEOUL, South Korea — North Korea’s leader, ..."
...,...,...,...,...
142565,An eavesdropping Uber driver saved his 16-year...,uber driver keith avila pick passeng look like...,218078,Uber driver Keith Avila picked up a p...
142566,Plane carrying six people returning from a Cav...,crew friday continu search lake eri plane carr...,218079,Crews on Friday continued to search L...
142567,After helping a fraction of homeowners expecte...,obama administr announc massiv effort help dis...,218080,When the Obama administration announced a...
142568,"Yes, this is real: Michigan just banned bannin...",stori updated new law michigan prohibit local ...,218081,This story has been updated. A new law in...


In [10]:
from datetime import datetime
query = "Corrupted politics"
corpus = df['title'].to_numpy()
start = datetime.now()
embedder = SentenceTransformer('msmarco-distilbert-base-v2')
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
first = datetime.now()

query_embedding = embedder.encode(query, convert_to_tensor=True)
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
top_results = torch.topk(cos_scores, k=5)
second = datetime.now()

# for score, idx in zip(top_results[0], top_results[1]):
#     print(df['title'][int(idx)], "(Score: {:.4f})".format(score))
print(second-first, first-start)
print(articles[result[1][0]])

0:00:00.018484 0:01:35.855487


NameError: name 'articles' is not defined

In [11]:
articles = df['article']

In [12]:
for score, idx in zip(top_results[0], top_results[1]):
    print(df['title'][int(idx)], "(Score: {:.4f})".format(score))

The Criminalization of Politics (Score: 0.5517)
Opposing a Corrupt Transaction (Score: 0.5377)
How Democrats Win Debates by Corrupting English (Score: 0.5348)
Who’s Really Corrupting Politics with Huge Gobs of Money? (Score: 0.5284)
The corruption cloud over Mayor de Blasio just got darker (Score: 0.5074)


In [13]:
start = datetime.now()
articles = df['article'].to_numpy()
vectorizer = TfidfVectorizer()
out = vectorizer.fit_transform(articles)
svd = TruncatedSVD(n_components=30)
out_reduced = svd.fit_transform(out).astype('float32',casting='same_kind')
index = faiss.IndexFlatL2(out_reduced.shape[1])
index.add(out_reduced)
first = datetime.now()
q = vectorizer.transform(np.asarray(["Corrupted politics"]))
q_reduced = svd.transform(q).astype('float32',casting='same_kind')
result = index.search(q_reduced, k=5)
second = datetime.now()
print(second-first, first-start)
result

0:00:00.039559 0:01:45.294552


(array([[0.00018736, 0.0002044 , 0.00020459, 0.0002074 , 0.0002074 ]],
       dtype=float32),
 array([[ 87650, 102677,  62671,  42038,  43408]]))

In [21]:
len(vectorizer.get_feature_names())

308324

In [23]:
words = array(vectorizer.get_feature_names())

In [22]:
from numpy import array, log
for row in out:
    row = row.toarray()
    row = log(row + 1)
    for k in tqdm(dict_idf.keys()):
    if k in words:
        tfidf[:, words == k] = tfidf[:, words == k] * dict_idf[k]
    pbar.update(1)
    break

[[0. 0. 0. ... 0. 0. 0.]]


In [14]:
q = vectorizer.transform(np.asarray(["Hello world"]))
q_reduced = svd.transform(q).astype('float32',casting='same_kind')
result = index.search(q_reduced, k=5)
print(result, articles[result[1][0]])

(array([[0.00091256, 0.001177  , 0.001574  , 0.00158769, 0.00173432]],
      dtype=float32), array([[ 40706, 112317,  46831, 137698,  99859]])) ['activ volcano exist world pictur recent eruptions'
 'twenti year ago today world first clone mammal born sheep name dolly'
 'long sorri histori of' 'tour earli america alexi de tocquevil marvel'
 'ekow eshun role art let us examin awkard cultur find thing bind us togeth']
